In [1]:
import numpy as np
from utils_predictions_manipulation import*
from utils_nans_manipulation import*
from cross_validation import*
from utils_data_loading import*
from utils_features_manipulation import*
import random

### Loading data

In [70]:
traindata,_ = load_data('Data/train.csv')

X_total, Y_total = structure_data(traindata)

X_total = replace_bad_data_with_nans(X_total, -999)

X_total, col = replace_nans_with_median(X_total, threshold=0.5)

#X_total = X_total + np.random.rand(X_total.shape[0], X_total.shape[1])/10000

index_list = list(range(X_total.shape[1]))
# 1 must be at the begining of degree_test
degree_test = [1, 1/30, 1/20, 1/10, 1/5, 1/4, 1/3, 1/2, 2, 3, 4, 5, 10, 20, 30]
degree = [degree_test for i in index_list]
index = [1,23]

X_poly = build_poly_index(X_total, index, degree)

BP: 1 [1, 0.03333333333333333, 0.05, 0.1, 0.2, 0.25, 0.3333333333333333, 0.5, 2, 3, 4, 5, 10, 20, 30]
BP: 23 [1, 0.03333333333333333, 0.05, 0.1, 0.2, 0.25, 0.3333333333333333, 0.5, 2, 3, 4, 5, 10, 20, 30]


In [72]:
thresh = [[23, 0.5],
          [23, 1.5],
          [23, 2.5],
          [12, 0],
          [13, 0.5]]

#Accounts for indexing 
mult = len(degree_test)-1
for split in thresh:
    split[0] = split[0] + sum([i<split[0] for i in index])*mult

X_sets, Y_sets, thresholds = split_data_set(X_poly, Y_total, thresh)

In [73]:
points_x = []
points_y = []
for i in range(len(X_sets)):
    X_set = X_sets[i]
    Y_set = Y_sets[i]
    
    acc = 0
    acc2 = 0
    print(thresholds[i])
    for j in range(len(X_set)):
        X_split = X_set[j]
        remove = []
        # Removes linearly dependent rows
        for i in range(1,X_split.shape[1]):
            if len(np.unique(X_split[:,i])) == 1:
                remove.append(i)
        X_split = np.delete(X_split,remove,1)
        
        Y_split = Y_set[j]
        
        dtmp_tr,dtmp_te = cross_validation(Y_split, X_split, k_fold=4, seed=1, function_name='least_squares')
        
        acc = acc + dtmp_tr*(X_split.shape[0]/X_total.shape[0])
        
        if dtmp_tr>0.3:
            acc2 = acc2 + dtmp_tr*(X_split.shape[0]/X_total.shape[0])
        else:
            select = max([sum(Y_split/Y_split.shape[0]), 1-sum(Y_split/Y_split.shape[0])])
            acc2 = acc2 + select*(Y_split.shape[0]/Y_total.shape[0])
            
        print(X_split.shape[0], "\t", dtmp_tr, dtmp_te, "\t", sum(Y_split/Y_split.shape[0])+dtmp_tr)
        points_x.append(sum(Y_split/Y_split.shape[0]))
        points_y.append(dtmp_tr)
    print("TOTAL:", acc)
    print("ARTIF:", acc2, "\n")

([37, 0.5],)
150087 	 0.7784796069756492 0.7790837131206524 	 1.179413731850194
99913 	 0.8334167400645902 0.8324725758667627 	 1.0885587135815071
TOTAL: 0.8004353420889107
ARTIF: 0.8004353420889107 

([37, 1.5],)
150087 	 0.7784796069756492 0.7790837131206524 	 1.179413731850194
99913 	 0.8334167400645902 0.8324725758667627 	 1.0885587135815071
TOTAL: 0.8004353420889107
ARTIF: 0.8004353420889107 

([37, 2.5],)
150087 	 0.7784796069756492 0.7790837131206524 	 1.179413731850194
99913 	 0.8334167400645902 0.8324725758667627 	 1.0885587135815071
TOTAL: 0.8004353420889107
ARTIF: 0.8004353420889107 

([26, 0],)
123934 	 0.7957777921225617 0.795412000129103 	 1.1354585899666012
126066 	 0.7948819647163345 0.7946519228328468 	 1.1404866479776308
TOTAL: 0.7953260586113879
ARTIF: 0.7953260586113879 

([27, 0.5],)
250000 	 0.7946133333333334 0.794772 	 1.1372813333335934
TOTAL: 0.7946133333333334
ARTIF: 0.7946133333333334 

([37, 0.5], [37, 1.5])
150087 	 0.7784796069756492 0.7790837131206524 	 

75411 	 0.7883336869651318 0.7876220029705071 	 1.1924657101445473
49258 	 0.8362636024037681 0.8356545395484813 	 1.0879790597914183
50655 	 0.8307799626207586 0.8304114348890468 	 1.089253953342345
TOTAL: 0.8065567861392808
ARTIF: 0.8065567861392808 



In [8]:
X_split

array([[ 1.00000000e+00,  1.43905000e+02,  8.14170000e+01, ...,
        -7.87171377e-04, -3.33072684e-02,  0.00000000e+00],
       [ 1.00000000e+00,  1.75864000e+02,  1.69150000e+01, ...,
        -9.95903504e-04, -3.32091839e-02,  0.00000000e+00],
       [ 1.00000000e+00,  1.05594000e+02,  5.05590000e+01, ...,
        -4.05559186e-04, -3.26799804e-02,  0.00000000e+00],
       ...,
       [ 1.00000000e+00,  1.12899865e+02,  5.81790000e+01, ...,
        -3.19209875e-05, -3.29227703e-02,  0.00000000e+00],
       [ 1.00000000e+00,  9.49510000e+01,  1.93620000e+01, ...,
        -1.65108134e-04, -3.32206927e-02,  0.00000000e+00],
       [ 1.00000000e+00,  1.12736057e+02,  7.27560000e+01, ...,
         5.33563617e-04, -3.26961510e-02,  0.00000000e+00]])

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(points_x, points_y)

In [ ]:
np.linalg.matrix_rank(X_sets[0][1])

array([[ 1.00000000e+00,  8.14170000e+01,  8.09430000e+01, ...,
         6.49864986e+01, -7.87171377e-04, -3.33072684e-02],
       [ 1.00000000e+00,  1.69150000e+01,  1.34805000e+02, ...,
         6.52619361e+01, -9.95903504e-04, -3.32091839e-02],
       [ 1.00000000e+00,  5.05590000e+01,  1.00989000e+02, ...,
         6.56959650e+01, -4.05559186e-04, -3.26799804e-02],
       ...,
       [ 1.00000000e+00,  5.81790000e+01,  6.80830000e+01, ...,
         6.59747928e+01, -3.19209875e-05, -3.29227703e-02],
       [ 1.00000000e+00,  1.93620000e+01,  6.88120000e+01, ...,
         6.49681829e+01, -1.65108134e-04, -3.32206927e-02],
       [ 1.00000000e+00,  7.27560000e+01,  7.08310000e+01, ...,
         6.58942862e+01,  5.33563617e-04, -3.26961510e-02]])

In [30]:

for i in range(X_split.shape[1]):
    print(X_split[:,i])
    print(len(np.unique(X_split[:,i])))

[1. 1. 1. ... 1. 1. 1.]
1
[143.905      175.864      105.594      ... 112.89986547  94.951
 112.73605694]
82470
[81.417 16.915 50.559 ... 58.179 19.362 72.756]
65105
[ 80.943 134.805 100.989 ...  68.083  68.812  70.831]
63577
[ 0.414 16.405  4.288 ... 22.439 13.504  7.479]
33282
[3.31  3.891 2.904 ... 2.585 3.365 2.025]
4340
[ 0.414 16.405  4.288 ... 22.439 13.504  7.479]
33287
[75.968 57.983 65.333 ... 50.618 55.859 83.24 ]
50327
[2.354 1.056 0.675 ... 1.162 0.999 0.936]
3449
[-1.285 -1.385 -1.366 ... -1.345  1.414 -1.411]
2757
[22.647 28.209 39.008 ... 23.416 27.944 43.003]
35756
[-1.655 -2.197  2.433 ... -1.609 -2.211  1.685]
4958
[ 0.01  -2.231 -2.532 ...  2.776  2.792  2.653]
6285
[53.321 29.774 26.325 ... 27.202 27.915 40.236]
38131
[-0.522  0.798  0.21  ...  0.308 -0.874  1.49 ]
4973
[-3.1    1.569  1.884 ...  1.042 -0.296  0.637]
6285
[31.082  2.723 37.791 ... 46.737 12.15  40.729]
50313
[ 0.06  -0.871  0.024 ... -0.867  0.811 -1.596]
6285
[ 86.062  53.131 129.804 ...  80.408 1